# ID2214/FID3214 Assignment 2 Group no. [7]
### Project members: 
**Xinlong Han** *xinlong@kth.se*
**Weiqi Xie** *weiqix@kth.se*
**Tianyu Deng** *tianyud@kth.se*

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [24]:
import numpy as np
import pandas as pd
import time

In [25]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.12.0
NumPy version: 1.26.1
Pandas version: 2.1.3


## Reused functions from Assignment 1

In [26]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_column_filter(df):
    copydf = df.copy()

    column_filter = []

    for column in copydf.columns:
        if column not in ["CLASS", "ID"]:
            unique_values = copydf[column].dropna().unique()

            if len(unique_values) > 1:
                column_filter.append(column)
            else:
                copydf = copydf.drop(column, axis=1)

    column_filter.extend(["CLASS", "ID"])

    return copydf, column_filter

def apply_column_filter(df, column_filter):
    copydf = df.copy()

    columns_to_keep = np.intersect1d(copydf.columns, column_filter)
    copydf = copydf[columns_to_keep]

    return copydf

def create_normalization(df, normalizationtype="minmax"):
    copydf = df.copy()
    normalization = {}

    for column in copydf.select_dtypes(include=['float', 'int']).columns:
        if column not in ["CLASS", "ID"]:
            col_values = copydf[column].values

            if normalizationtype == "minmax":
                min_value = np.min(col_values)
                max_value = np.max(col_values)

                copydf[column] = (col_values - min_value) / (max_value - min_value)

                normalization[column] = ("minmax", min_value, max_value)

            elif normalizationtype == "zscore":
                mean_value = np.mean(col_values)
                std_value = np.std(col_values)

                copydf[column] = (col_values - mean_value) / std_value

                normalization[column] = ("zscore", mean_value, std_value)

    return copydf, normalization

def apply_normalization(df, normalization):
    copydf = df.copy()

    for column, (norm_type, param1, param2) in normalization.items():
        col_values = copydf[column].values

        if norm_type == "minmax":
            copydf[column] = (col_values - param1) / (param2 - param1)
        elif norm_type == "zscore":
            copydf[column] = (col_values - param1) / param2

    return copydf

def create_imputation(df):
    df_copy = df.copy()
    imputation = {}

    for col in df_copy.columns:
        if col not in ['ID', 'CLASS']:
            if df[col].dtype == 'float' or df[col].dtype == 'int': 
                mean_val = df[col].mean()
                df_copy[col] = df_copy[col].fillna(mean_val)
                imputation[col] = mean_val
            elif df[col].dtype == 'object' or df[col].dtype.name == 'category': 
                mode_val = df[col].mode()[0]
                df_copy[col] = df_copy[col].fillna(mode_val)
                imputation[col] = mode_val
    return df_copy, imputation

def apply_imputation(df, imputation):
    df_copy = df.copy()

    for col, value in imputation.items():
        df_copy[col] = df_copy[col].fillna(value)

    return df_copy

def create_bins(df, nobins=10, bintype='equal-width'):
    df_copy = df.copy()
    binning = {}

    for col in df_copy.columns:
        if col not in ['ID', 'CLASS'] and df_copy[col].dtype in ['float', 'int']:
            if bintype == 'equal-width':
                df_copy[col], bins = pd.cut(df_copy[col], nobins, retbins=True, labels=False, duplicates='drop')
            elif bintype == 'equal-size':
                df_copy[col], bins = pd.qcut(df_copy[col], nobins, retbins=True, labels=False, duplicates='drop')
            binning[col] = bins
            binning[col][0] = -np.inf
            binning[col][-1] = np.inf
            df_copy[col] = df_copy[col].astype('category')
            df_copy[col] = df_copy[col].cat.set_categories([str(i) for i in df_copy[col].cat.categories], rename=True)
        else:
            df_copy[col] = df_copy[col].astype('category')

    return df_copy, binning

def apply_bins(df, binning):
    df_copy = df.copy()

    for col, bins in binning.items():
        df_copy[col] = pd.cut(df_copy[col], bins, labels=False)
        df_copy[col] = df_copy[col].astype('category')
        df_copy[col] = df_copy[col].cat.set_categories([str(i) for i in df_copy[col].cat.categories], rename=True)

    return df_copy

def split(df, testfraction=0.5):
    rand_indices = np.random.permutation(df.index)
    split_indices = int(len(rand_indices) * testfraction)

    test_indices = rand_indices[:split_indices]
    train_indices = rand_indices[split_indices:]
    test_df = df.loc[test_indices]
    train_df = df.loc[train_indices]

    return train_df, test_df

def accuracy(predictions, correctlabels):

    predictedlabels = predictions.idxmax(axis=1)
    acc = sum(predictedlabels == correctlabels) / len(predictions)

    return acc

def create_one_hot(df):
    df_copy = df.copy()
    one_hot = {}
    
    selected_columns = df_copy.select_dtypes(include=['object', 'category']).items()

    for col, value in selected_columns:
        if col not in ['CLASS', 'ID']:
            unique_values = value.unique()
            one_hot[col] = unique_values

            for val in unique_values:
                new_col_name = f'{col}_{str(val)}'
                df_copy[new_col_name] = (value == val).astype('float')

            df_copy.drop(col, axis=1, inplace=True)

    return df_copy, one_hot

def apply_one_hot(df, one_hot):
    df_copy = df.copy()

    for col, value in one_hot.items():
        for val in value:
            new_col_name = f'{col}_{str(val)}'
            df_copy[new_col_name] = (df_copy[col] == val).astype('float')

        df_copy.drop(col, axis=1, inplace=True)

    return df_copy

def folds(dataframe,nofolds=10):
    if nofolds <= 1:
        raise ValueError("Nofolds need to be greater then 1.")
    copydf = dataframe.copy()
    np.random.permutation(copydf.index) 
    folds_list = []
    
    for i in range(nofolds):
        start_idx = int(len(copydf) * i / nofolds)
        end_idx = int(len(copydf) * (i + 1) / nofolds)
        folds_list.append(copydf[start_idx:end_idx])
    
    return folds_list

def brier_score(dataframe, correct_labels):
    df = dataframe.copy()
    correct_df = pd.get_dummies(correct_labels)
    brier_score = np.mean(np.sum((df - correct_df)**2, axis=1))
    return brier_score

def auc_binary(predictions, correctlabels, threshold, c):
    # Extract relevant data for the specified class c
    correctlabels_class = np.array(correctlabels) == predictions.columns[c]
    predictions_class = predictions[predictions.columns[c]]
    
    # Calculate True Positive Rate (tpr) and False Positive Rate (fpr)
    predicted_labels = predictions_class[correctlabels_class] >= threshold
    pos = sum(predicted_labels)
    tpr = pos / sum(correctlabels_class)
    
    # Repeat the process for the negative class
    not_correctlabels_class = np.array(correctlabels) != predictions.columns[c]
    predictions_class = predictions[predictions.columns[c]]
    predicted_labels = predictions_class[not_correctlabels_class] >= threshold
    neg = sum(predicted_labels)
    fpr = neg / sum(not_correctlabels_class)
    
    return tpr, fpr


def auc(predictions, correctlabels):
    # Unique thresholds based on predictions
    thresholds = np.unique(predictions)
    AUC_d = {}
    
    # Iterate over all classes and calculate the area under the ROC curve (AUC)
    for (index, c) in enumerate(np.unique(correctlabels)):
        # Calculate ROC points for the current class and thresholds
        roc_points = [auc_binary(predictions, correctlabels, th, index) for th in reversed(thresholds)]
        
        # Calculate AUC as the area under the curve
        AUC = 0
        tpr_last = 0
        fpr_last = 0
        
        # Iterate over all thresholds
        for r in roc_points:
            tpr, fpr = r
            # Add area under the triangle
            if tpr > tpr_last and fpr > fpr_last:
                AUC += (fpr - fpr_last) * tpr_last + (fpr - fpr_last) * (tpr - tpr_last) / 2
            # Add area under the rectangle
            elif fpr > fpr_last:
                AUC += (fpr - fpr_last) * tpr
            # Update point coordinates (tpr, fpr) of the curve
            tpr_last = tpr
            fpr_last = fpr
        
        AUC_d[c] = AUC
    
    # Calculate the weighted average for all classes
    AUC_total = 0
    for (cName, auc) in AUC_d.items():
        number_of_labels = np.sum(np.array(correctlabels) == cName)
        weight = number_of_labels / len(correctlabels)
        AUC_total += weight * auc
    
    return AUC_total

## 1. Define the class kNN

In [27]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies



In [28]:
class kNN:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        self.training_time = None

    def fit(self, df, normalizationtype='minmax'):
        copydf = df.copy()

        copydf, self.column_filter = create_column_filter(copydf)
        copydf, self.imputation = create_imputation(copydf)
        copydf, self.normalization = create_normalization(copydf, normalizationtype)
        copydf, self.one_hot = create_one_hot(copydf)
        
        self.training_labels = copydf['CLASS'].astype('category')
        self.labels = list(self.training_labels.cat.categories)
        self.training_data = copydf[copydf.columns.difference(['CLASS', 'ID'])].to_numpy()

    def distance(self, point1, point2):
        return np.linalg.norm(point1 - point2)

    def prediction(self, x_test, k):
        distances = np.empty(self.training_data.shape[0])

        for index, row in enumerate(self.training_data):
            distances[index] = self.distance(row, x_test)

        sorted_indicies = distances.argsort()
        k_indicies = sorted_indicies[:k]
        k_labels = self.training_labels[k_indicies]
        unique, counts = np.unique(k_labels, return_counts=True)
        probabilities = dict(zip(unique, counts/k))

        return probabilities

    def predict(self, df, k=5):
        copydf = df.copy()

        copydf = apply_column_filter(copydf, self.column_filter)
        copydf = apply_imputation(copydf, self.imputation)
        copydf = apply_normalization(copydf, self.normalization)
        copydf = apply_one_hot(copydf, self.one_hot)

        labels = np.unique(self.training_labels)
        columns = copydf[df.columns.difference(['CLASS', 'ID'])]
        values = (columns.select_dtypes(include=np.number)).to_numpy()

        result = pd.DataFrame(0.0, index=range(0, len(self.training_data)), columns=labels)

        for index, row in enumerate(values):
            probabilities = self.prediction(row, k)
            for prob in probabilities:
                result.at[index, prob] = probabilities[prob]

        return result

In [29]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.00 s.
Testing time (k=1): 0.05 s.
Testing time (k=3): 0.04 s.
Testing time (k=5): 0.04 s.
Testing time (k=7): 0.04 s.
Testing time (k=9): 0.04 s.


'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.474019,0.833805
7,0.598131,0.470723,0.834465
9,0.616822,0.483674,0.828734


In [30]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [34]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.

class NaiveBayes:
    def __init__(self):
        self.binning = None
        self.priors = None
        self.feature_value_counts = None
        self.feature_counts = None
        self.labels = None

    def fit(self, dataframe, nobins=10, bintype="equal-width"):
        df_temp = dataframe.copy()
        df_temp, self.binning = create_bins(df_temp,nobins,bintype)
        df_temp["CLASS"] = df_temp["CLASS"].astype("category")
        self.labels = list(df_temp["CLASS"].cat.categories)
        self.priors = dict(df_temp["CLASS"].value_counts(normalize = True))
        feature_class_value_counts = {}
        feature_class_counts = {}
        for col in df_temp.columns:
            if col not in ["CLASS", "ID"]:
                df_temp = df_temp.dropna(axis = 0, how="any", subset = ["CLASS", col])
                feature_class_counts[col] = dict(df_temp["CLASS"].value_counts())
                g = df_temp.groupby(["CLASS", col], observed=True).size()
                feature_class_value_counts[col] = dict(g)
        self.feature_counts = feature_class_counts
        self.feature_value_counts = feature_class_value_counts

    def predict(self, dataframe):
        df = dataframe.copy()
        df = apply_bins(df, self.binning)
        labels = self.labels
        df.drop(columns=["ID","CLASS"],inplace=True)
        nrow, ncol, nlabel = df.shape[0], df.shape[1], len(labels)
        matrix = np.zeros([nlabel, nrow, ncol])
        for col_num in range(ncol):
            col = df.columns[col_num]
            for label_num in range(nlabel):
                label = labels[label_num]
                for row_num in range(nrow):
                    value = df.iloc[row_num, col_num]
                    if((label, value) in self.feature_value_counts[col].keys()):
                        features_value_count = self.feature_value_counts[col][(label, value)]
                        feature_count = self.feature_counts[col][label]
                        relative_freq = features_value_count / feature_count
                    else:
                        relative_freq = 0
                    matrix[label_num, row_num, col_num] = relative_freq
        product = matrix.prod(axis=2)
        prior = np.array([self.priors[labels[i]] for i in range(nlabel)])
        prior = np.tile(prior, nrow).reshape(nrow,nlabel).T  # notice the shape
        prob =  product * prior
        sum_prob = prob.sum(axis=0)
        sum_zero = sum_prob==0.0
        sum_prob += sum_zero.astype('float')
        norm_prob = prob/sum_prob
        predictions = pd.DataFrame(norm_prob.T, columns = labels)
        return predictions


In [35]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.02 s.
Testing time (3, 'equal-width'): 0.07 s.
Training time (3, 'equal-size'): 0.02 s.
Testing time (3, 'equal-size'): 0.06 s.
Training time (5, 'equal-width'): 0.02 s.
Testing time (5, 'equal-width'): 0.07 s.
Training time (5, 'equal-size'): 0.02 s.
Testing time (5, 'equal-size'): 0.08 s.
Training time (10, 'equal-width'): 0.02 s.
Testing time (10, 'equal-width'): 0.07 s.
Training time (10, 'equal-size'): 0.02 s.
Testing time (10, 'equal-size'): 0.07 s.


'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.621325  0.723560
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.626168     0.559282  0.750656
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.598131     0.570270  0.747255
   equal-size   0.579439     0.743837  0.746409

In [36]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.